In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

sys.path.append(os.path.join(os.getcwd(), "../pykan"))
from kan import *

sys.path.append(os.path.join(os.getcwd(), "nuclr"))
import torch
from data import prepare_nuclear_data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from argparse import Namespace
from sklearn.model_selection import train_test_split

def build_semf_dataset(device="cpu"):
    # load args from disk
    args = {}   
    args["TARGETS_REGRESSION"] = {
        "binding_semf": 1,
    }

    args["TARGETS_CLASSIFICATION"] = {
    }
    args["TRAIN_FRAC"] = 1.0
    args["SEED"] = 0
    args["PER_NUCLEON"] = False
    args["TRAIN_SET"] = "all"
    args["DEV"] = device
    args = Namespace(**args)
    data = prepare_nuclear_data(args)

    # Get rid of NANs
    indices = (data.y == data.y).flatten()
    X = data.X[indices][:,0:2]
    y = data.y[indices]

    # Scale data
    scaler = MinMaxScaler()
    y_transformed = torch.tensor(scaler.fit_transform(y), device=device)
    
    train_input, test_input, train_label, test_label = train_test_split(X, y_transformed, test_size=0.2)

    dataset = {"train_input" : train_input, "test_input" : test_input, "train_label" : train_label, "test_label" : test_label, "scaler" : scaler}
    return dataset

dataset = build_semf_dataset(device="cpu")

In [3]:
# create a KAN: 2D inputs, 1D output, and 5 hidden neurons. cubic spline (k=3), 5 grid intervals (grid=5).
model = KAN(width=[2,16,16,16,1], grid=5, k=3, seed=0)

In [4]:
# train the model
model.train(dataset, opt="LBFGS", steps=200, lamb=0.01, lamb_entropy=10.)

description:   0%|                                                          | 0/200 [00:00<?, ?it/s]

train loss: 2.42e-01 | test loss: 2.48e-01 | reg: 3.01e+02 :   0%|  | 1/200 [00:13<44:39, 13.46s/it]


KeyboardInterrupt: 